In [13]:
import matplotlib.pyplot as plt
import networkx as nx
import sys
import pickle as pkl
import numpy as np
import hdbscan
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import sys
from functools import reduce
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score, accuracy_score
from tqdm import tqdm
from copy import deepcopy
from pprint import pprint
from networkx.algorithms import node_classification
from sklearn.model_selection import train_test_split


def parse_index_file(filename):
    """Parse index file."""
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index


def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)


def load_data(dataset_str):
    """
    Loads input data from gcn/data directory
    ind.dataset_str.x => the feature vectors of the training instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.tx => the feature vectors of the test instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.allx => the feature vectors of both labeled and unlabeled training instances
        (a superset of ind.dataset_str.x) as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.y => the one-hot labels of the labeled training instances as numpy.ndarray object;
    ind.dataset_str.ty => the one-hot labels of the test instances as numpy.ndarray object;
    ind.dataset_str.ally => the labels for instances in ind.dataset_str.allx as numpy.ndarray object;
    ind.dataset_str.graph => a dict in the format {index: [index_of_neighbor_nodes]} as collections.defaultdict
        object;
    ind.dataset_str.test.index => the indices of test instances in graph, for the inductive setting as list object.
    All objects above must be saved using python pickle module.
    :param dataset_str: Dataset name
    :return: All data input files loaded (as well the training/test data).
    """
    names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
    objects = []
    for i in range(len(names)):
        with open("datasets/data-gcn/ind.{}.{}".format(dataset_str, names[i]), 'rb') as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding='latin1'))
            else:
                objects.append(pkl.load(f))

    x, y, tx, ty, allx, ally, graph = tuple(objects)
    test_idx_reorder = parse_index_file("datasets/data-gcn/ind.{}.test.index".format(dataset_str))
    test_idx_range = np.sort(test_idx_reorder)

    if dataset_str == 'citeseer':
        # Fix citeseer dataset (there are some isolated nodes in the graph)
        # Find isolated nodes, add them as zero-vecs into the right position
        test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
        tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
        tx_extended[test_idx_range-min(test_idx_range), :] = tx
        tx = tx_extended
        ty_extended = np.zeros((len(test_idx_range_full), y.shape[1]))
        ty_extended[test_idx_range-min(test_idx_range), :] = ty
        ty = ty_extended

    features = sp.vstack((allx, tx)).tolil()
    features[test_idx_reorder, :] = features[test_idx_range, :]
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))

    labels = np.vstack((ally, ty))

    return adj, features, labels

def data_helper(dname):
    def sub_helper(G, keyname):
        labels = [G.nodes[i][keyname] for i in G.nodes]
        sorted_labels_set = sorted(list(set(labels)))
        labels_idx = np.array(list(map(lambda x: sorted_labels_set.index(x), labels)))
        return G, labels_idx
    if dname in ['cora', 'citeseer', 'pubmed']:
        adj, features, labels =load_data(dname)
        labels_idx = np.argmax(labels, axis=-1)
        G=nx.from_scipy_sparse_matrix(adj)
        return G, labels_idx
    elif dname == 'karate':
        G=nx.karate_club_graph()
        return sub_helper(G, 'club')
    elif dname == 'strike':
        G=nx.read_gml("datasets/real-classic/strike.gml")
        return sub_helper(G, 'value')
    elif dname == 'polbooks':
        G=nx.read_gml("datasets/real-classic/polbooks.gml")
        return sub_helper(G, 'value')
    elif dname == 'polblogs':
        G=nx.read_gml("datasets/real-classic/polblogs.gml")
        return sub_helper(G, 'value')
    elif dname == 'football':
        G=nx.read_gml("datasets/real-classic/football.gml")
        return sub_helper(G, 'value')
    elif 'lfr' in dname:
        mu = .1 * int(dname.split('_')[1])
        assert mu <= .9 and mu >= .1
        G = nx.algorithms.community.community_generators.LFR_benchmark_graph(
            1000, 3, 1.5, mu, average_degree=5, min_degree=None, max_degree=None, 
            min_community=20, max_community=None, tol=1e-07, max_iters=500, seed=None)
        for i in G:
            G.nodes[i]['community_tuple'] = tuple(sorted(list( G.nodes[i]['community'])))
        return sub_helper(G, 'community_tuple')
    
def partition2label(G, partition):
    c = 0
    node2idx = {}
    for i in G.nodes:
        node2idx[i] = c
        c += 1
    inferred_labels = np.zeros((reduce(lambda x, y: x + y, list(map(lambda z: len(z), partition)))))
    assert len(G) == len(inferred_labels)
    for ind, i in enumerate(partition):
        for j in i:
            inferred_labels[node2idx[j]] = ind
    return inferred_labels

In [2]:
all_graphs = {
    'lfr_mu_1': lambda: nx.algorithms.community.community_generators.LFR_benchmark_graph(
        1000, 3, 1.5, 0.1, average_degree=5, min_degree=None, max_degree=None, min_community=20, 
        max_community=None, tol=1e-07, max_iters=500, seed=None),
    'lfr_mu_5': lambda: nx.algorithms.community.community_generators.LFR_benchmark_graph(
        1000, 3, 1.5, 0.5, average_degree=5, min_degree=None, max_degree=None, min_community=20, 
        max_community=None, tol=1e-07, max_iters=500, seed=None),
    'lfr_mu_9': lambda: nx.algorithms.community.community_generators.LFR_benchmark_graph(
        1000, 3, 1.5, 0.9, average_degree=5, min_degree=None, max_degree=None, min_community=20, 
        max_community=None, tol=1e-07, max_iters=500, seed=None),
    'karate': lambda: nx.karate_club_graph(),
    'strike': lambda: nx.read_gml("datasets/real-classic/strike.gml"),
    'football': lambda: nx.read_gml("datasets/real-classic/football.gml"),
    'polbooks': lambda: nx.read_gml("datasets/real-classic/polbooks.gml"),
    
}

In [ ]:
strike, karate, polblog, polbooks, football

In [ ]:
performance = {}

for i in ['cora', 'citeseer', 'pubmed', 'karate', 'strike', 'polbooks', 'polblogs', 'football', 'lfr_3', 'lfr_6', 'lfr_9']:
    performance[i] = {
        'harmonic': {
            'test_ratio':{
                '.2': {'acc': []},
                '.05': {'acc': []},
                '.5': {'acc': []},
                '1000': {'acc': []},
                '.95': {'acc': []},
            }
        },
        'consist': {
             'test_ratio':{
                '.2': {'acc': []},
                '.05': {'acc': []},
                '.5': {'acc': []},
                '1000': {'acc': []},
                '.95': {'acc': []},
            }
        }
    }

for dname in tqdm(performance):
    print('=====', dname)
    for r in range(10):
        G, labels = data_helper(dname)
        X = list(G.nodes)
        y = labels
        
        if dname in ['cora', 'citeseer', 'pubmed']:
            test_sizes = [(1000, '1000')]
        elif dname in ['karate', 'strike', 'polbooks', 'polblogs', 'football']:
            test_sizes = [
                (.05, '.05'),
                (.5, '.5'),
                (.95, '.95')
            ]
        else:
            test_sizes = [(.2, '.2')]
        
        for ts, ts_key in test_sizes:
            X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
                X, y, np.array(list(range(len(y)))), test_size=ts, random_state=42)
            for ind, i in enumerate(X_train):
                G.node[i]['label'] = y_train[ind]

            all_inferred_labs = node_classification.harmonic_function(G)
            test_inferred_labs = []
            for i in idx_test:
                test_inferred_labs.append(all_inferred_labs[i])
            performance[dname]['harmonic']['test_ratio'][ts_key]['acc'].append(accuracy_score(y_true=y_test, y_pred=test_inferred_labs))

            all_inferred_labs = node_classification.local_and_global_consistency(G)
            test_inferred_labs = []
            for i in idx_test:
                test_inferred_labs.append(all_inferred_labs[i])
            performance[dname]['consist']['test_ratio'][ts_key]['acc'].append(accuracy_score(y_true=y_test, y_pred=test_inferred_labs))



  0%|                                                                                           | 0/11 [00:00<?, ?it/s]

===== cora


  9%|███████▌                                                                           | 1/11 [00:05<00:54,  5.41s/it]

===== citeseer
